### <div style="text-align:center">Daily Update</div>

Import new daily donation file from folder.

In [25]:
# importing required libraries
import pandas as pd
import mysql.connector
import pymysql
from mysql.connector import errorcode
from sqlalchemy import create_engine
import boto3
import s3fs
import datetime as dt
from datetime import datetime
import os


# Insert your path name to the folder where the new daily file is stored.(ex. 'C:\\abc\\\')
path = r'C:\\Users\\PC\\Desktop\\Donors_DW_Daily' 

os.chdir(path)
files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)

oldest = files[0]
newest = files[-1]

print ("Oldest:", oldest)
print ("Newest:", newest)
print ("All by modified oldest to newest:", files)


Oldest: Contributions 4.27.2021.csv
Newest: Contributions 4.27.2021.csv
All by modified oldest to newest: ['Contributions 4.27.2021.csv']


In [26]:
#Read to csv and make sure the data looks as expected.

nd = pd.read_csv(path+'\\'+newest)

nd

,FakeID,PrimaryAffiliation,PersonOrg,City,State,Zip,FiscalYear,ContributionDate,Amount,ContributionType,DesignationCode,DesignationName,VSEPurpose,College,DesignationType,DesignationPurpose,PaymentType
0,832,Alum,P,Teaneck,NJ,7666,2021,4/27/2021,100,Gift,AF00,Yeshiva University Annual Fund,3A01 - Unrestricted,Yeshiva University,Unrestricted,Annual Fund,Lockbox Holding
1,4428,Parent of Past Student,P,Silver Spring,MD,20910,2021,4/27/2021,"1,000",Gift,GU08,RIETS General Unrestricted Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Lockbox Holding
2,4938,Parent of Past Student,P,Bronx,NY,10463,2021,4/27/2021,500,Gift,GU00,Yeshiva University General Unrestricted Fund,3A01 - Unrestricted,Yeshiva University,Unrestricted,Annual Fund,Credit Card (VISA)
3,7938,Alum,P,Teaneck,NJ,7666,2021,4/27/2021,"1,000",Gift,GS03,YUHS for Boys General Scholarship Fund,NaN,YU Boys High School,Restricted,Annual Fund,Lockbox Holding
4,8910,Parent of Past Student,P,New Rochelle,NY,10804,2021,4/27/2021,100,Gift,AF08,RIETS Annual Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Lockbox Holding
5,10711,Friend,P,Englewood Cliffs,NJ,7632,2021,4/27/2021,108,Gift,GU08,RIETS General Unrestricted Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Lockbox Holding
6,10791,Alum,P,San Diego,CA,92115,2021,4/27/2021,30,Gift,GU08,RIETS General Unrestricted Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Cash or Check
7,17279,Alum,P,Teaneck,NJ,7666,2021,4/27/2021,100,Gift,F971,Rabbinic Alumni Services - Dues and Operations,3A02 - Academic Divisions,RIETS,Restricted,NaN,Lockbox Holding
8,12186,Alum,P,Lawrence,NY,11559,2021,4/27/2021,"3,000",Gift,GU08,RIETS General Unrestricted Fund,3A02 - Academic Divisions,RIETS,Unrestricted,Annual Fund,Cash or Check
9,302,Alum,P,Teaneck,NY,7666,2021,4/27/2021,"1,300",Pledge Payment,F971,Rabbinic Alumni Services - Dues and Operations,3A02 - Academic Divisions,RIETS,Restricted,NaN,Lockbox Holding


Next, clean the daily donor data set.

Change the colleges 'YU Girls High School' and 'YU Boys High School' to 'High Schools (combined)'

In [27]:
nd['College'] = nd['College'].replace(['YU Girls High School','YU Boys High School'],
                                    ['High Schools (combined)','High Schools (combined)'])
                                                                                 
nd['College'].value_counts()

RIETS                      7
Yeshiva University         4
High Schools (combined)    1
Name: College, dtype: int64

Combine the college 'Undergraduate Men' with 'Undergraduate'

In [28]:
nd['College'] = nd['College'].replace(['Undergraduate Men'],['Undergraduate'])
                                                                                 
nd['College'].value_counts()

RIETS                      7
Yeshiva University         4
High Schools (combined)    1
Name: College, dtype: int64

Remove any payment type called "Goods".  This refers to payments collected for goods & services which is a used to cover food at an event and cannot be counted as a charitable donation. 

In [29]:
discard = ["Goods"]
nd2 = nd[~nd['PaymentType'].str.contains('|'.join(discard))]
print (nd2.shape)
nd2['PaymentType'].value_counts()

(12, 17)


Lockbox Holding       8
Cash or Check         2
Wire Transfer         1
Credit Card (VISA)    1
Name: PaymentType, dtype: int64

Combine payment types where appropriate.<br>
1) Combine Cash/Check into Cash or Cash.<br> 
2) Combine Credit Card (VISA), Credit Card (Master Card), Credit Card (Discover), Credit Card (American Express) into Credit Card. <br>
3) Combine Bequest Wired into Bequest Realized.<br> 
4) Combine Matching Wire Transfer into Matching Gift. <br>
5) Combine Canadian Friends of YU into Cash or Check. <br>
6) Combine Israel Retained Cash into Cash or Check. <br>
7) Combine Canadian Cash into Cash or Check. <br>
8) Combine Stock into Stocks and Securities. <br>
9) Combine Electronic Funds Transfer into Wire Transfer.<br>

In [30]:
nd2['PaymentType'] = nd2['PaymentType'].replace(['Cash/Check','Credit Card (VISA)','Credit Card (Master Card)',
                                                 'Credit Card (Discover)','Credit Card (American Express)','Bequest Wired',
                                                 'Matching Wire Transfer','Canadian Friends of YU', 'Israel Retained Cash',
                                                'Canadian Cash','Stock','Electronic Funds Transfer'],
                                                ['Cash or Check','Credit Card','Credit Card','Credit Card','Credit Card',
                                                 'Bequest Realized','Matching Gift','Cash or Check','Cash or Check',
                                                 'Cash or Check','Stocks and Securities','Wire Transfer'])
                                               
                                               
nd2['PaymentType'].value_counts()

Lockbox Holding    8
Cash or Check      2
Credit Card        1
Wire Transfer      1
Name: PaymentType, dtype: int64

## Sending Dataframe to S3 Bucket (Archive)

In [31]:
# creating a variable that stores date to be append on the file to be archive
today = dt.datetime.today()
date = today.strftime("%Y%m%d")

In [32]:
#connect and load the historical data file to s3

s3pathName = 's3://yu-donation-bucket/'
FileNameFullPath = 'donations' 
filenames3 = "%s%s%s.csv"%(s3pathName,FileNameFullPath,date) #name of the filepath and csv file

nd2.to_csv(filenames3, header=True, line_terminator='\n') 

#print success message
print("Successfull uploaded file to location:"+str(filenames3))

Successfull uploaded file to location:s3://yu-donation-bucket/donations20210506.csv


## Connect to MySQL Workbench

In [33]:
# credentials for connecting to MySQL

host_name = "database-1.cvt3j4afpbrp.us-east-1.rds.amazonaws.com"
db_name = "staging_db"
u_name = "admin"
pwd = "Joli2016!"

In [34]:
# create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=db_name, user=u_name, pw=pwd))

# connect to server
connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')
print('Connected to database.')
cursor = connection.cursor()

Connected to database.


### Loading Donation Staging Table

In [35]:
# call store procedure to delete data from the donation staging table
cursor.callproc('staging_db.del_donation')
print("Donation staging table (donation_stg) cleared.")

Donation staging table (donation_stg) cleared.


In [36]:
# loading the donation staging table (donation_stg) in the staging database
connection.commit()
nd2.to_sql('donation_stg', engine, if_exists='append', index=False)
print("Donation staging table (donation_stg) loaded.")


Donation staging table (donation_stg) loaded.


### Loading Donation Table in the Donation Database

In [37]:
# connect to donation_db
donation_db = "donation_db"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=host_name, db=donation_db, user=u_name, pw=pwd))

connection = mysql.connector.connect(user=u_name, password=pwd,host= host_name, port ='3306')

print('Connected to database.')
cursor = connection.cursor()

Connected to database.


In [38]:
# call stored procedures to load dimenstions in donation_db
cursor.callproc('donation_db.load_dim_donation')
print("Donation dimension table (dim_donation) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_date')
print("Date dimension table (dim_date) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_donor')
print("Donor dimension table (dim_donor) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_college')
print("College dimension table (dim_college) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_designation')
print("Designation dimension table (dim_designation) in donation database (donation_db) loaded.")

cursor.callproc('donation_db.load_dim_loc')
print("Location dimension table (dim_location) in donation database (donation_db) loaded.")

#call stored procedures to load fact table in donation_db
cursor.callproc('donation_db.load_donation_fact')
print("Donation fact table (donation_fact) in donation database (donation_db) loaded.")


Donation dimension table (dim_donation) in donation database (donation_db) loaded.
Date dimension table (dim_date) in donation database (donation_db) loaded.
Donor dimension table (dim_donor) in donation database (donation_db) loaded.
College dimension table (dim_college) in donation database (donation_db) loaded.
Designation dimension table (dim_designation) in donation database (donation_db) loaded.
Location dimension table (dim_location) in donation database (donation_db) loaded.
Donation fact table (donation_fact) in donation database (donation_db) loaded.


In [39]:
# commiting and closing database connection
connection.commit()
connection.close()
print('Disconnected from database.')

Disconnected from database.
